In [33]:
import dedupe
import json
from cytoolz import *
from copy import copy
import re
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from urllib.parse import urlparse
from unicodedata import normalize

In [34]:
fields = [{'field': 'description', 'type': 'String', 'has missing': True},
         {'field': 'price', 'type': 'Price', 'has missing': True}]


def tokenize_url(url):
    path = urlparse(url).path
    path = path.replace('/', ' ')
    path = path.replace('-', ' ')
    return word_tokenize(path)


def remove_accents(input_str):
    return normalize('NFKD', input_str).encode('ascii', 'ignore').decode()


def sanitize_string(input_str):
    pattern = r'[?|&|!|@|#|;|*|~|(|)|´|^|\r|\n|\t]'
    return clean_string(re.sub(pattern, r'', remove_accents(input_str.lower())))


def clean_string(input_str):
    unuseful = set(['confira', 'compre', 'oferta', 'novo',
                    'preco', 'agora', 'melhores', 'aqui',
                    'aproveite', 'menor', 'maior', 'encontra',
                    'site', 'condicoes', 'ofertas', 'imbativeis',
                    'vendas', 'online', 'novo', 'nova', 'tecnologia',
                    'precos', 'pagamento', 'melhor', 'veja', 'encontre',
                    'menores', 'vem', 'venha', 'ver'])

    names = set(['walmart', 'walmart.com', 'shoptime', 'shoptime.com',
                 'pontofrio', 'pontofrio.com', 'magazineluiza', 'magazineluiza.com',
                 'magazine luiza', 'americanas', 'americanas.com', 'submarino',
                 'submarino.com', 'extra', 'extra.com.br', 'casas bahia', 'casasbahia.com.br'])

    stopwords = [remove_accents(s) for s in nltk.corpus.stopwords.words('portuguese')]
    repl = list(concat([unuseful, names, stopwords]))
    tk = word_tokenize(input_str)
    return ' '.join([word for word in tk if word not in repl])


def preprocess(item):
    return {'description': sanitize_string(item['description']), 'price': float(sanitize_string(item['price']))}

In [35]:
with open('../data/walmart/buscape.json') as f:
    walmart = [preprocess(dissoc(item, 'image')) for item in json.loads(f.read())['products']]

with open('../data/submarino/buscape.json') as f:
    submarino = [preprocess(dissoc(item, 'image')) for item in json.loads(f.read())['products']]

In [36]:
p = {str(i): walmart[i] for i in range(len(walmart))}
w = {str(i): submarino[i] for i in range(len(submarino))}

In [37]:
linker = dedupe.RecordLink(fields)
linker.sample(p, w)
dedupe.consoleLabel(linker)

description : livro - design grafico integracao autodesk 3ds max 2010 adobe photoshop cs4 extended - aprenda recursos fundamentais forma pratica visual - 9788536502601
price : 109.6

description : brinco prata zirconias 11564 4064732
price : 89.0

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


n


description : blusa sacada estampada
price : 129.99

description : boneca princesas disney - mini princesa amigo - costuras cinderela b5333 - hasbro 2621211
price : 59.99

0/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


description : bico injetor gm vectra 8v 2.2 1998 2003 monza kadett mpfi gasolina 1991 1996 s10 blazer 2.2 199
price : 140.45

description : bico injetor gm vectra 8v 2.2 1998 2003 monza kadett mpfi gasolina 1991 1996 s10 blazer 2.2 1996 1997 3171632
price : 140.44

0/10 positive, 2/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

description : sissy bar destacavel cav . fantasma - shadow 750 2010 - jjcustom
price : 818.99

description : sissy bar destacavel cav . fantasma hd dyna super glide 1600 2008 2014 jjcustom 2873316
price : 894.47

10/10 positive, 21/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


description : camisa fox - 180 race
price : 139.9

description : farol palio siena strada g3 03 04 05 06 07 08 09 10 11 12 13 14 15 mascara cromada foco duplo lado direito passageiro 3867679
price : 139.9

10/10 positive, 22/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


In [38]:
linker.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.100000
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (3, description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (4, description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (1, description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (2, description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.8, description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.4, description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.2, description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.6, description)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.6, description)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.2, description)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.4, description)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.8,

In [39]:
m = linker.match(p,w, 0)

INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.6, description)
INFO:dedupe.api:0 records
INFO:dedupe.api:100 records


In [40]:
m

[(('3999', '3944'), 0.79641318),
 (('3787', '1531'), 0.75663102),
 (('738', '302'), 0.69547784),
 (('2872', '795'), 0.67398232),
 (('2869', '797'), 0.66353774),
 (('4001', '3951'), 0.62680578),
 (('3510', '2253'), 0.60751826),
 (('3800', '1537'), 0.54060543),
 (('3491', '2262'), 0.51300508),
 (('3803', '1519'), 0.40277994),
 (('2849', '2269'), 0.40034854),
 (('3488', '2266'), 0.36752567),
 (('2887', '1217'), 0.36571729),
 (('3489', '2252'), 0.33275783),
 (('2861', '1454'), 0.33249581),
 (('3497', '2254'), 0.32417876),
 (('3809', '1523'), 0.30562451),
 (('4541', '3552'), 0.30531472),
 (('3503', '2247'), 0.2779274),
 (('4559', '3144'), 0.25304466),
 (('3502', '2259'), 0.25260404),
 (('3496', '2263'), 0.25094247),
 (('3507', '2250'), 0.2412139),
 (('2797', '2618'), 0.23926325),
 (('741', '1909'), 0.2219941),
 (('1494', '1923'), 0.22037077),
 (('3495', '2265'), 0.21026333),
 (('3798', '1521'), 0.20818588),
 (('4555', '3152'), 0.17429905),
 (('3792', '1518'), 0.17184785),
 (('3085', '3877')

In [41]:
for i in m:
    print(p[i[0][0]], w[i[0][1]])

{'description': 'bico injetor gm vectra 8v 2.2 1998 2003 monza kadett mpfi gasolina 1991 1996 s10 blazer 2.2 1996 1997 3171632', 'price': 140.44} {'description': 'bico injetor gm vectra 8v 2.2 1998 2003 monza kadett mpfi gasolina 1991 1996 s10 blazer 2.2 199', 'price': 140.45}
{'description': 'fantasia borboleta basic arca noe 1882969', 'price': 79.99} {'description': 'fantasia borboleta basic', 'price': 75.99}
{'description': 'poltrona decorativa sala escritorio tilla sued amarelo - jm estofados 2475857', 'price': 299.0} {'description': 'poltrona decorativa sala escritorio tilla sued preto - jm estofados', 'price': 299.0}
{'description': 'cozinha completa maia 05 palmeira moveis jacarta esmeralda 3578314', 'price': 1699.88} {'description': 'cozinha completa maia 05 palmeira moveis jacarta / esmeralda', 'price': 1699.9}
{'description': 'cozinha completa maia 02 palmeira moveis jacarta preto 3578309', 'price': 1099.9} {'description': 'cozinha completa maia 02 palmeira moveis jacarta / p

In [42]:
len(walmart)

4569

In [43]:
len(submarino)

4527